In [1]:
#Some basic libraries

import numpy as np
import pandas as pd
import gc
import math
import re
import datetime

#Web scraping imports 

import requests
from bs4 import BeautifulSoup as bs
import json

# For Viz

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

# Scraping gun law by state

In [2]:
# Load in the webpage 


r = requests.get('https://www.gunstocarry.com/gun-laws-state/#states')

# Convert to a Beautiful Soup Object 

soup = bs(r.content)

# Create Variable for just the body

guns_carry_body = soup.body

In [3]:
# Creating Variable for Main Container 

gc_main_container = guns_carry_body.find('div', class_= "e5507-53 x-container max width")

In [4]:
# Grabbing al of the state names 

state_names_1 = gc_main_container.find_all('span' , class_="x-acc-header-text") # Change to find for LOOP

# Grabbing boxes with all rest of info BOX_STATE CONATINS two Tables for each state 

box_state = gc_main_container.find_all('div', class_="x-acc-content")

#Cretaing Variable for ONLY state_info_real 

box_state_info = gc_main_container.find_all('tbody', class_="row-hover")

# CREATING LIST OF STATE NAMES

In [5]:
# Loop for grabbing state names and CREATING LIST OF STATE NAMES

state_name_list = []

for index, x in enumerate(state_names_1):
    state_name_list.append(x.text)

len(state_name_list)

102

In [6]:
# Pairing down to 1 iteration (51 states)

sname_list = state_name_list[:51]

len(sname_list)

51

# CREATING A LIST OF LISTS for law types (for each state)

In [7]:
type_law_list = []

for i in range(102):
    con = []
    for x in box_state_info[i].find_all('td', class_="column-1"):
        texty = x.get_text()
        con.append(texty)
    type_law_list.append(con)

In [8]:
# Pairing down to 51 states 

law_list = type_law_list[:51]

law_list

[['Permit required',
  'Purchase permit required',
  'Registration of firearm',
  'Open carry allowed',
  'Background checks on private gun sales required'],
 ['Permit required',
  'Purchase permit required',
  'Registration of firearm',
  'Open carry allowed',
  'Background checks on private gun sales required'],
 ['Permit required',
  'Purchase permit required',
  'Registration of firearm',
  'Open carry allowed',
  'Background checks on private gun sales required',
  'Magazine size restriction'],
 ['Permit required',
  'Purchase permit required',
  'Registration of firearm',
  'Open carry allowed',
  'Background checks on private gun sales required',
  'Magazine size restriction'],
 ['Permit required',
  'Purchase permit required',
  'Registration of firearm',
  'Open carry allowed',
  'Background checks on private gun sales required',
  'Magazine size restriction'],
 ['Permit required',
  'Purchase permit required',
  'Registration of firearm',
  'Open carry allowed',
  'Background

### Creating list for column names - ALL POSSIBLE LAW NAMES 

In [9]:
df_column_names = ['Permit required','Carry permit required',
  'Purchase permit required',
  'Registration of firearm',
  'Open carry allowed',
  'Background checks on private gun sales required',
  'Magazine size restriction',
  'Assault weapon law',
  'Owner permit required',
  'Red flag law',
  'Peaceable journey law',
  'Restriction on NFA weapons',
  'Ammunition restrictions',
  'State Preemption of local restrictions', 'Notes']

#### Testing to see if all possible names are the above list 

In [10]:
# making sure all possible column names are present 

for x in law_list:
    for words in x:
        if words not in df_column_names:
            print(words, 'NOT PRESENT')

Carry Permit required NOT PRESENT
Concealed carry permit required NOT PRESENT
Permit needed for open carry? NOT PRESENT
State preemption NOT PRESENT
Assault weapons banned  NOT PRESENT
Concealed carry permit required NOT PRESENT
Owner license required NOT PRESENT


#### The above check revealed some missing names 
#### The above check revealed a capitilaztion issue
#### Fixing the cap issue 
#### Also removing the question mark after - Permit needed for open carry?
#### Also removing the space after Assault weapons banned(space was here)

In [11]:
# CLEANING UP textual issues in law_list

typ_la_lis = []

for i in range(102):
    cox = []
    for x in box_state_info[i].find_all('td', class_="column-1"):
        tex = x.get_text()
        if tex == 'Carry Permit required':
            tex = tex.replace('Carry Permit required', 'Carry permit required')
        if tex == 'Assault weapons banned ':
            tex = tex.replace('Assault weapons banned ', 'Assault weapons banned')
        if tex == 'Permit needed for open carry?':
            tex = tex.replace('Permit needed for open carry?', 'Permit needed for open carry')
        cox.append(tex)
    typ_la_lis.append(cox)
    
law_list_up1 = typ_la_lis[:51]

#### Checking to make sure the updates worked WE KNOW THEY WORKED ON THE Permit to permit already

In [12]:
bad_ones = ['Assault weapons banned ','Permit needed for open carry?']

for x in law_list_up1:
    for wor in x:
        if wor in bad_ones:
            print(wor , 'IS HERE MOFO')
            
# NO RESULT is SUCCESS 

### law_list_up1 is active law list WITHOUT SPECIFIC ADDITIONS 

#### Adding the missing names to the list and creating var - df_column_names_2

In [13]:
missed_laws = []

# making list of all missing law types 

for x in law_list_up1:
    for word in x:
        if word not in df_column_names:
            #print(word)
            missed_laws.append(word)

# missed_laws is a list of missing laws
# Adding missed_laws to df_column_names 
# That will be df_column_names_2

df_column_names_2 = df_column_names + missed_laws

#df_column_names_2

#### Ordering df_column_name list and removing bad syntax by hand since its easiest way 

In [14]:
# ASSAULT WEAPON BANNED has a space after it 

# Permit needed for carry has a QUESTION MARK in it 

df_column_names_ordered =[
 'Permit required',
 'Carry permit required',
 'Concealed carry permit required',
 'Permit needed for open carry',
 'Concealed carry permit required',
 'Purchase permit required',
 'Owner permit required',
 'Owner license required',
 'Registration of firearm',
 'Open carry allowed',
 'Background checks on private gun sales required',
 'Magazine size restriction',
 'Assault weapon law',
 'Red flag law',
 'Peaceable journey law',
 'Restriction on NFA weapons',
 'Ammunition restrictions',
 'State Preemption of local restrictions',
 'State preemption',
 'Assault weapons banned',
 'Notes']

In [15]:
#df_column_names_ordered

# Print test is success 

# THE LIST OF POTENTIAL LAW_NAMES syntactically matches with the ACTUAL STATES LIST OF LISTS

#### CREATE LIST WITH ALL TYPES appended by handgun (REMOVING NOTES from column list for use reasons)

In [16]:
handgun_columns = [x + ' hand-gun' for x in df_column_names_ordered if x not in 'Notes']

In [17]:
handgun_columns

['Permit required hand-gun',
 'Carry permit required hand-gun',
 'Concealed carry permit required hand-gun',
 'Permit needed for open carry hand-gun',
 'Concealed carry permit required hand-gun',
 'Purchase permit required hand-gun',
 'Owner permit required hand-gun',
 'Owner license required hand-gun',
 'Registration of firearm hand-gun',
 'Open carry allowed hand-gun',
 'Background checks on private gun sales required hand-gun',
 'Magazine size restriction hand-gun',
 'Assault weapon law hand-gun',
 'Red flag law hand-gun',
 'Peaceable journey law hand-gun',
 'Restriction on NFA weapons hand-gun',
 'Ammunition restrictions hand-gun',
 'State Preemption of local restrictions hand-gun',
 'State preemption hand-gun',
 'Assault weapons banned hand-gun']

#### CREATE LIST WITH ALL TYPES appended by handgun (REMOVING NOTES from column list for use reasons)

In [18]:
longgun_columns = [x + ' long-gun' for x in df_column_names_ordered if x not in 'Notes']

In [19]:
longgun_columns

['Permit required long-gun',
 'Carry permit required long-gun',
 'Concealed carry permit required long-gun',
 'Permit needed for open carry long-gun',
 'Concealed carry permit required long-gun',
 'Purchase permit required long-gun',
 'Owner permit required long-gun',
 'Owner license required long-gun',
 'Registration of firearm long-gun',
 'Open carry allowed long-gun',
 'Background checks on private gun sales required long-gun',
 'Magazine size restriction long-gun',
 'Assault weapon law long-gun',
 'Red flag law long-gun',
 'Peaceable journey law long-gun',
 'Restriction on NFA weapons long-gun',
 'Ammunition restrictions long-gun',
 'State Preemption of local restrictions long-gun',
 'State preemption long-gun',
 'Assault weapons banned long-gun']

#### CREATING MASTER LIST OF COLUMN NAMES (Minus Notes Column for Now)

In [20]:
# There is no reason to stagger the names and group them next to each other. THIS DF WILL ALWAYS BE ANALYZED BY CODE DUE TO SIZE 

master_column_name_list = handgun_columns + longgun_columns

master_column_name_list

['Permit required hand-gun',
 'Carry permit required hand-gun',
 'Concealed carry permit required hand-gun',
 'Permit needed for open carry hand-gun',
 'Concealed carry permit required hand-gun',
 'Purchase permit required hand-gun',
 'Owner permit required hand-gun',
 'Owner license required hand-gun',
 'Registration of firearm hand-gun',
 'Open carry allowed hand-gun',
 'Background checks on private gun sales required hand-gun',
 'Magazine size restriction hand-gun',
 'Assault weapon law hand-gun',
 'Red flag law hand-gun',
 'Peaceable journey law hand-gun',
 'Restriction on NFA weapons hand-gun',
 'Ammunition restrictions hand-gun',
 'State Preemption of local restrictions hand-gun',
 'State preemption hand-gun',
 'Assault weapons banned hand-gun',
 'Permit required long-gun',
 'Carry permit required long-gun',
 'Concealed carry permit required long-gun',
 'Permit needed for open carry long-gun',
 'Concealed carry permit required long-gun',
 'Purchase permit required long-gun',
 'Ow

# CREATING A LIST OF LISTS for Handgun answers

In [21]:
handG_list = []

for i in range(102):
    coni = []
    for x in box_state_info[i].find_all('td', class_="column-2"):
        textyi = x.get_text()
        coni.append(textyi)
    handG_list.append(coni)

In [22]:
# Pairing down to 51 states 

handgun_states_list = handG_list[:51]

handgun_states_list

[['Yes', 'No', 'No', 'Yes', 'No'],
 ['No', 'No', 'No', 'Yes', 'No'],
 ['No', 'No', 'No', 'Yes', 'No', 'No'],
 ['No', 'No', 'No', 'Yes', 'No', 'No'],
 ['Yes', 'Yes', 'Yes', 'Yes*', 'Yes', 'Yes'],
 ['Yes', 'No', 'No', 'Yes*', 'Yes', 'Yes'],
 ['Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes'],
 ['Yes', 'No', 'No', 'Yes', 'Yes', 'No'],
 ['Yes', 'Yes', 'Yes', 'No', 'Yes', 'Yes'],
 ['Yes', 'No', 'No', 'No', 'No', 'No'],
 ['Yes', 'No', 'No', 'Yes', 'No', 'No'],
 ['Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes'],
 ['No*', 'No', 'No', 'Yes', 'No', 'No'],
 ['Yes', 'Yes', 'No', 'No', 'Yes', 'No'],
 ['Yes', 'No', 'No', 'Yes', 'No', 'No'],
 ['Yes', 'Yes', 'No', 'Yes', 'Yes', 'No'],
 ['No', 'No', 'No', 'Yes', 'No', 'No'],
 ['Yes', 'No', 'No', 'Yes', 'No', 'No'],
 ['Yes', 'No', 'No', 'Yes', 'No', 'No'],
 ['No', 'No', 'No', 'Yes', 'No', 'No'],
 ['Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes'],
 ['Yes', 'Yes', 'No', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes'],
 ['Yes', 'No*', 'Yes', 'Yes', 'Yes', 'No'],
 ['Yes',
  'Ye

# CREATING LIST OF LISTS for long gun answers

In [23]:
longG_list = []

for i in range(102):
    conit = []
    for x in box_state_info[i].find_all('td', class_="column-3"):
        textyit = x.get_text()
        conit.append(textyit)
    longG_list.append(conit)

In [24]:
# Pairing down to 51 states 

longgun_states_list = longG_list[:51]

# CREATING LIST of LISTS for Notes

In [25]:
notes_list = []

for i in range(102):
    coniti = []
    for x in box_state_info[i].find_all('td', class_="column-4"):
        textyiti = x.get_text()
        coniti.append(textyiti)
    notes_list.append(coniti)

In [26]:
# Pairing down to 51 states

notes_states_list = notes_list[:51]

# CREATING DICTIONARY and LIST for import into DF

In [27]:
# STACK OVERFLOW LINK TO CREATING DF FROM DICTIONARIES 

# https://stackoverflow.com/questions/33157522/create-pandas-dataframe-from-dictionary-of-dictionaries

In [28]:
hand_li_for_di = []

for x in range(51):
    di_di = {}
    keys_newH = []
    keys5 = law_list_up1[x]
    for laH in keys5:
        laH = laH + ' hand-gun'
        keys_newH.append(laH)
    values5 = handgun_states_list[x]
    di_di = dict(zip(keys_newH, values5))
    hand_li_for_di.append(di_di)

In [29]:
# THIS IS A LIST of dictionaries with keys(that are law type) and VALUES(that are hand-gun answers)

hand_li_for_di

[{'Permit required hand-gun': 'Yes',
  'Purchase permit required hand-gun': 'No',
  'Registration of firearm hand-gun': 'No',
  'Open carry allowed hand-gun': 'Yes',
  'Background checks on private gun sales required hand-gun': 'No'},
 {'Permit required hand-gun': 'No',
  'Purchase permit required hand-gun': 'No',
  'Registration of firearm hand-gun': 'No',
  'Open carry allowed hand-gun': 'Yes',
  'Background checks on private gun sales required hand-gun': 'No'},
 {'Permit required hand-gun': 'No',
  'Purchase permit required hand-gun': 'No',
  'Registration of firearm hand-gun': 'No',
  'Open carry allowed hand-gun': 'Yes',
  'Background checks on private gun sales required hand-gun': 'No',
  'Magazine size restriction hand-gun': 'No'},
 {'Permit required hand-gun': 'No',
  'Purchase permit required hand-gun': 'No',
  'Registration of firearm hand-gun': 'No',
  'Open carry allowed hand-gun': 'Yes',
  'Background checks on private gun sales required hand-gun': 'No',
  'Magazine size r

In [30]:
long_li_for_di = []

for x in range(51):
    di_diL = {}
    keys_newL = []
    keys6 = law_list_up1[x]
    for laL in keys6:
        laL = laL + ' long-gun'
        keys_newL.append(laL)
    values6 = longgun_states_list[x]
    di_diL = dict(zip(keys_newL, values6))
    long_li_for_di.append(di_diL)

In [31]:
# THIS IS A LIST of dictionaries with keys(that are law type) and VALUES(that are long-gun answers)

long_li_for_di

[{'Permit required long-gun': 'No',
  'Purchase permit required long-gun': 'No',
  'Registration of firearm long-gun': 'No',
  'Open carry allowed long-gun': 'Yes',
  'Background checks on private gun sales required long-gun': 'No'},
 {'Permit required long-gun': 'No',
  'Purchase permit required long-gun': 'No',
  'Registration of firearm long-gun': 'No',
  'Open carry allowed long-gun': 'Yes',
  'Background checks on private gun sales required long-gun': 'No'},
 {'Permit required long-gun': 'No',
  'Purchase permit required long-gun': 'No',
  'Registration of firearm long-gun': 'No',
  'Open carry allowed long-gun': 'Yes',
  'Background checks on private gun sales required long-gun': 'No',
  'Magazine size restriction long-gun': 'No'},
 {'Permit required long-gun': 'No',
  'Purchase permit required long-gun': 'No',
  'Registration of firearm long-gun': 'No',
  'Open carry allowed long-gun': 'Yes',
  'Background checks on private gun sales required long-gun': 'No',
  'Magazine size re

### MAKING COPIES of dictionary lists to append 

In [32]:
#Hand-gun
li_hand_law_copy = hand_li_for_di.copy()

#Long-gun
li_long_law_copy = long_li_for_di.copy()

In [33]:
# Merging them together 

#both_laws_dici = {}

both_laws_list = []

for w in range(51):
    both_laws_dici = {}
    both_laws_dici = {**li_hand_law_copy[w], **li_long_law_copy[w]}
    both_laws_list.append(both_laws_dici)

In [34]:
both_laws_list

[{'Permit required hand-gun': 'Yes',
  'Purchase permit required hand-gun': 'No',
  'Registration of firearm hand-gun': 'No',
  'Open carry allowed hand-gun': 'Yes',
  'Background checks on private gun sales required hand-gun': 'No',
  'Permit required long-gun': 'No',
  'Purchase permit required long-gun': 'No',
  'Registration of firearm long-gun': 'No',
  'Open carry allowed long-gun': 'Yes',
  'Background checks on private gun sales required long-gun': 'No'},
 {'Permit required hand-gun': 'No',
  'Purchase permit required hand-gun': 'No',
  'Registration of firearm hand-gun': 'No',
  'Open carry allowed hand-gun': 'Yes',
  'Background checks on private gun sales required hand-gun': 'No',
  'Permit required long-gun': 'No',
  'Purchase permit required long-gun': 'No',
  'Registration of firearm long-gun': 'No',
  'Open carry allowed long-gun': 'Yes',
  'Background checks on private gun sales required long-gun': 'No'},
 {'Permit required hand-gun': 'No',
  'Purchase permit required h

## Creating a dictionary where state names are keys and both_law_list dicts are values 

In [35]:
# Copy of the list of dicts

both_laws_list_COPY = both_laws_list.copy()

In [36]:
both_laws_list_COPY

[{'Permit required hand-gun': 'Yes',
  'Purchase permit required hand-gun': 'No',
  'Registration of firearm hand-gun': 'No',
  'Open carry allowed hand-gun': 'Yes',
  'Background checks on private gun sales required hand-gun': 'No',
  'Permit required long-gun': 'No',
  'Purchase permit required long-gun': 'No',
  'Registration of firearm long-gun': 'No',
  'Open carry allowed long-gun': 'Yes',
  'Background checks on private gun sales required long-gun': 'No'},
 {'Permit required hand-gun': 'No',
  'Purchase permit required hand-gun': 'No',
  'Registration of firearm hand-gun': 'No',
  'Open carry allowed hand-gun': 'Yes',
  'Background checks on private gun sales required hand-gun': 'No',
  'Permit required long-gun': 'No',
  'Purchase permit required long-gun': 'No',
  'Registration of firearm long-gun': 'No',
  'Open carry allowed long-gun': 'Yes',
  'Background checks on private gun sales required long-gun': 'No'},
 {'Permit required hand-gun': 'No',
  'Purchase permit required h

## Here are the Master LIST and Master DICTIONARY 

In [37]:
# Creating master list of dictionaries 

master_list_of_d = []
master_dict_of_d = {}

for x in range(51):
    blank_d = {}
    key = sname_list[x]
    value = both_laws_list_COPY[x]
    blank_d[key] = {}
    blank_d[key] = value 
    master_list_of_d.append(blank_d)
    master_dict_of_d.update(blank_d)

In [38]:
master_list_of_d

[{'Alabama': {'Permit required hand-gun': 'Yes',
   'Purchase permit required hand-gun': 'No',
   'Registration of firearm hand-gun': 'No',
   'Open carry allowed hand-gun': 'Yes',
   'Background checks on private gun sales required hand-gun': 'No',
   'Permit required long-gun': 'No',
   'Purchase permit required long-gun': 'No',
   'Registration of firearm long-gun': 'No',
   'Open carry allowed long-gun': 'Yes',
   'Background checks on private gun sales required long-gun': 'No'}},
 {'Alaska': {'Permit required hand-gun': 'No',
   'Purchase permit required hand-gun': 'No',
   'Registration of firearm hand-gun': 'No',
   'Open carry allowed hand-gun': 'Yes',
   'Background checks on private gun sales required hand-gun': 'No',
   'Permit required long-gun': 'No',
   'Purchase permit required long-gun': 'No',
   'Registration of firearm long-gun': 'No',
   'Open carry allowed long-gun': 'Yes',
   'Background checks on private gun sales required long-gun': 'No'}},
 {'Arizona': {'Permit 

In [39]:
master_dict_of_d

{'Alabama': {'Permit required hand-gun': 'Yes',
  'Purchase permit required hand-gun': 'No',
  'Registration of firearm hand-gun': 'No',
  'Open carry allowed hand-gun': 'Yes',
  'Background checks on private gun sales required hand-gun': 'No',
  'Permit required long-gun': 'No',
  'Purchase permit required long-gun': 'No',
  'Registration of firearm long-gun': 'No',
  'Open carry allowed long-gun': 'Yes',
  'Background checks on private gun sales required long-gun': 'No'},
 'Alaska': {'Permit required hand-gun': 'No',
  'Purchase permit required hand-gun': 'No',
  'Registration of firearm hand-gun': 'No',
  'Open carry allowed hand-gun': 'Yes',
  'Background checks on private gun sales required hand-gun': 'No',
  'Permit required long-gun': 'No',
  'Purchase permit required long-gun': 'No',
  'Registration of firearm long-gun': 'No',
  'Open carry allowed long-gun': 'Yes',
  'Background checks on private gun sales required long-gun': 'No'},
 'Arizona': {'Permit required hand-gun': 'No

# TURNING IT ALL INTO A DF

In [40]:
gun_law_df = pd.DataFrame(master_dict_of_d)

In [41]:
gun_law_df_EXPORTING = gun_law_df.T

### Below is commented out so exporting does not happen each time 

In [42]:
# gun_law_df_EXPORTING.to_excel("gun_laws_master.xlsx") 

# THIS IS ALL RANDOM WORK BELOW

#### Creating MASTER LIST of lists (each state laws = list)

In [43]:
# Creating a MASTER LIST of LISTS (eash individual list contains the gun_law specs for each state)

master_list = []
for i in box_state_info:
    tesa = []
    tesa.append(re.findall('[A-Z][^A-Z]*', i.text.replace('\n', '')))
    master_list.append(tesa)

In [44]:
master_list[0]

[['Permit required',
  'Yes',
  'No',
  'Policy is ',
  'Shall ',
  'Issue ',
  'Purchase permit required',
  'No',
  'No',
  'Registration of firearm',
  'No',
  'No',
  'Open carry allowed',
  'Yes',
  'Yes',
  'Firearm must be in holster',
  'Background checks on private gun sales required',
  'No',
  'No']]

#### TESTING VARIABLES HERE

In [45]:
testing_box1 = box_state[0]

In [46]:
box_state_info[0]

<tbody class="row-hover">
<tr class="row-2 even">
<td class="column-1">Permit required</td><td class="column-2">Yes</td><td class="column-3">No</td><td class="column-4">Policy is Shall Issue </td>
</tr>
<tr class="row-3 odd">
<td class="column-1">Purchase permit required</td><td class="column-2">No</td><td class="column-3">No</td><td class="column-4"></td>
</tr>
<tr class="row-4 even">
<td class="column-1">Registration of firearm</td><td class="column-2">No</td><td class="column-3">No</td><td class="column-4"></td>
</tr>
<tr class="row-5 odd">
<td class="column-1">Open carry allowed</td><td class="column-2">Yes</td><td class="column-3">Yes</td><td class="column-4">Firearm must be in holster</td>
</tr>
<tr class="row-6 even">
<td class="column-1">Background checks on private gun sales required</td><td class="column-2">No</td><td class="column-3">No</td><td class="column-4"></td>
</tr>
</tbody>

#### KEEPING FOR EXAMPLE OF re.findall

In [47]:
ma_list = []
for r in box_state_info:
    te = []
    te.append(re.findall('[A-Z][^A-Z]*', r.text.replace('\n', '')))
    ma_list.append(te)